In [96]:
import datetime

import pandas as pd
import numpy as np

import sqlite3
import os

In [97]:
conn = sqlite3.connect('../db.sqlite3')

In [98]:
c = conn.cursor()

In [99]:
conn = sqlite3.connect('../db.sqlite3')

cursorObj = conn.cursor()
cursorObj.execute('SELECT name from sqlite_master where type= "table"')
print(cursorObj.fetchall())

c.close()

[('django_migrations',), ('sqlite_sequence',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_user',), ('auth_group',), ('django_session',), ('weekly_planner_recipe',), ('weekly_planner_ingredients',), ('weekly_planner_recipe_fact',)]


# Get Data

In [122]:
# Import data
df = pd.read_excel('scraped_recipes_clean.xlsx')

In [123]:
# Convert quantities to mono-dose
df['quantity'] = np.rint(df['quantity'].astype(float) / df['dose'])

# Ingredients

In [124]:
def existing_ingredients(db = '../db.sqlite3'):
    
    conn = sqlite3.connect(db)
    c = conn.cursor()
    
    # Create table
    c.execute('''SELECT * FROM weekly_planner_ingredients''')
    
    cols = ['ingredient', 'timestamp', 'unit_measure']
    df = pd.DataFrame(c.fetchall(), columns = cols)
    conn.close()
    
    return df

In [125]:
# Check Existing Ingredients
ext = existing_ingredients()
new_ingredients = df[['ingredient', 'unit_measure', 'timestamp']].drop_duplicates()

# Remove Ingredients that are not in the new data
new_df = new_ingredients[~new_ingredients['ingredient'].isin(ext['ingredient'])]

assert len(new_df) == len(new_df['ingredient'].unique())

In [126]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in new_df.itertuples():

    c.execute("""INSERT INTO weekly_planner_ingredients 
                ('Ingredient', 'Timestamp', 'Unit_measure')
                VALUES ('{}', '{}', '{}')""".format(i.ingredient, i.timestamp, i.unit_measure)) 

conn.commit()
conn.close()

# Recipes

In [127]:
rec = df[['recipe_name', 'recipe_source', 'url', 'timestamp']]
rec = rec.drop_duplicates()

In [128]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in rec.itertuples():

    c.execute("""INSERT INTO weekly_planner_recipe 
                VALUES ('{}', '{}', '{}', '{}')""".format(i.recipe_name, 
                                                          i.recipe_source, 
                                                          i.url,
                                                          i.timestamp)) 

conn.commit()
conn.close()

# Recipe Facts

In [129]:
facts = df[["quantity", "timestamp", "ingredient", "recipe_name"]]
facts = facts.drop_duplicates()

len(facts)

478

In [130]:
conn = sqlite3.connect('../db.sqlite3')
c = conn.cursor()

for i in facts.itertuples():

    c.execute("""INSERT INTO weekly_planner_recipe_fact 
                (quantity, timestamp, ingredient, recipe_name)
                VALUES ('{}', '{}', '{}', '{}')""".format(i.quantity, 
                                                          i.timestamp, 
                                                          i.ingredient,
                                                          i.recipe_name)) 

conn.commit()
conn.close()

In [141]:
t = 1201

In [133]:
t % 50

29

In [142]:
t + 50 - (t % 50)

1250